In [ ]:
import dautil as dl
import numpy as np
import pandas as pd
import ch7util
import matplotlib.pyplot as plt
from scipy.stats import norm
from IPython.display import HTML

In [ ]:
context = dl.nb.Context('non_parametric')
lr = dl.nb.LatexRenderer(chapter=7, start=6, context=context)
lr.render(r'=\mu=\frac{2\ N_+\ N_-}{N} + 1')
lr.render(r'\sigma^2=\frac{2\ N_+\ N_-\ (2\ N_+\ N_--N)}{N^2\ (N-1)}=\frac{(\mu-1)(\mu-2)}{N-1}')
lr.render(r'Z = \frac{R - \mu}{\sigma}')

In [ ]:
def count_runs(signs):
    nruns = 0
    prev = None

    for s in signs:
        if s != 0 and s != prev:
            nruns += 1

        prev = s

    return nruns

In [ ]:
def proc_runs(symbol):
    ohlc = dl.data.OHLC()
    close = ohlc.get(symbol)['Adj Close'].values
    diffs = np.diff(close)
    nplus = (diffs > 0).sum()
    nmin = (diffs < 0).sum()
    n = nplus + nmin
    mean = (2 * (nplus * nmin) / n) + 1
    var = (mean - 1) * (mean - 2) / (n - 1)
    std = np.sqrt(var)
    signs = np.sign(diffs)
    nruns = count_runs(np.diff(signs))

    return mean, std, (nruns - mean) / std

In [ ]:
means = []
stds = []
zscores = []

for symbol in ch7util.STOCKS:
    mean, std, zscore = proc_runs(symbol)
    means.append(mean)
    stds.append(std)
    zscores.append(zscore)

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 1, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 1, context)
dl.plotting.plot_text(sp.ax, means, stds, ch7util.STOCKS, add_scatter=True)
sp.label()

dl.plotting.bar(sp.next_ax(), ch7util.STOCKS, zscores)
sp.ax.axhline(norm.ppf(0.95), label='95 % confidence level')
sp.label()
HTML(sp.exit())